## Deliverable 2. Create a Customer Travel Destinations Map.

In [5]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [6]:
import numpy as np

In [30]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
#city_data_df = pd.read_csv("Weather_Database/WeatherPy_Database.csv")
city_data_df.head()



,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,San Andres,CO,12.5847,-81.7006,82.42,89,40,4.61,scattered clouds
1,1,Bredasdorp,ZA,-34.5322,20.0403,64.72,53,88,7.47,overcast clouds
2,2,Jamestown,US,42.0970,-79.2353,72.95,74,75,9.22,broken clouds
3,3,Cidreira,BR,-30.1811,-50.2056,66.81,77,80,17.54,broken clouds
4,4,Cape Town,ZA,-33.9258,18.4232,69.12,54,20,11.50,few clouds


In [38]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximun temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 40
What is the maximun temperature you would like for your trip? 90


In [39]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,San Andres,CO,12.5847,-81.7006,82.42,89,40,4.61,scattered clouds
1,1,Bredasdorp,ZA,-34.5322,20.0403,64.72,53,88,7.47,overcast clouds
2,2,Jamestown,US,42.0970,-79.2353,72.95,74,75,9.22,broken clouds
3,3,Cidreira,BR,-30.1811,-50.2056,66.81,77,80,17.54,broken clouds
4,4,Cape Town,ZA,-33.9258,18.4232,69.12,54,20,11.50,few clouds
5,5,Bodden Town,KY,19.2833,-81.2500,80.60,83,40,4.61,scattered clouds
6,6,Trincomalee,LK,8.5711,81.2335,82.40,76,66,22.64,broken clouds
7,7,Vila Do Maio,CV,15.1333,-23.2167,86.79,80,95,12.57,overcast clouds
8,8,Hermanus,ZA,-34.4187,19.2345,66.79,65,13,5.79,few clouds
9,9,East London,ZA,-33.0153,27.9116,68.94,64,0,17.27,clear sky


In [40]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                556
City                   556
Country                552
Lat                    556
Lng                    556
Max Temp               556
Humidity               556
Cloudiness             556
Wind Speed             556
Current Description    556
dtype: int64

In [44]:
preferred_cities_df["Country"].count()

552

In [55]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [56]:
clean_df.count()

City_ID                552
City                   552
Country                552
Lat                    552
Lng                    552
Max Temp               552
Humidity               552
Cloudiness             552
Wind Speed             552
Current Description    552
dtype: int64

In [57]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,San Andres,CO,82.42,scattered clouds,12.5847,-81.7006,
1,Bredasdorp,ZA,64.72,overcast clouds,-34.5322,20.0403,
2,Jamestown,US,72.95,broken clouds,42.0970,-79.2353,
3,Cidreira,BR,66.81,broken clouds,-30.1811,-50.2056,
4,Cape Town,ZA,69.12,few clouds,-33.9258,18.4232,
5,Bodden Town,KY,80.60,scattered clouds,19.2833,-81.2500,
6,Trincomalee,LK,82.40,broken clouds,8.5711,81.2335,
7,Vila Do Maio,CV,86.79,overcast clouds,15.1333,-23.2167,
8,Hermanus,ZA,66.79,few clouds,-34.4187,19.2345,
9,East London,ZA,68.94,clear sky,-33.0153,27.9116,


In [58]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"] 
    except (IndexError):
        print("Hotel not found...skipping.")

Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...sk

In [78]:
hotels

{'html_attributions': [],
 'results': [{'business_status': 'OPERATIONAL',
   'geometry': {'location': {'lat': -21.0465431, 'lng': 31.66655149999999},
    'viewport': {'northeast': {'lat': -21.0453673197085,
      'lng': 31.66809238029149},
     'southwest': {'lat': -21.0480652802915, 'lng': 31.6653944197085}}},
   'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png',
   'icon_background_color': '#909CE1',
   'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet',
   'name': 'Clevers Hotel Chiredzi',
   'opening_hours': {'open_now': True},
   'photos': [{'height': 3024,
     'html_attributions': ['<a href="https://maps.google.com/maps/contrib/113895834593155921674">Partners Chiriseri</a>'],
     'photo_reference': 'Aap_uEAt2HDeTlco1vDyiKl3J1dSTcEAMFfisGNVfL6fP1yj97VAaodcdqHdevdunnuWmuXO2DD21pzr_-7BebvDvVSFPWTlbdqp5ERQsxg8ilk0XSs90uNLGIoWsHnLQIZQcxeUxoh6IPelZP2fzaedZa1v1WrEZ1YySLVvNNELYCOFXMIU',
     'width': 4032}],
   '

In [59]:
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,San Andres,CO,82.42,scattered clouds,12.5847,-81.7006,Hotel Arena Blanca
1,Bredasdorp,ZA,64.72,overcast clouds,-34.5322,20.0403,Bredasdorp Country Manor
2,Jamestown,US,72.95,broken clouds,42.0970,-79.2353,Hampton Inn & Suites Jamestown
3,Cidreira,BR,66.81,broken clouds,-30.1811,-50.2056,Hotel Castelo
4,Cape Town,ZA,69.12,few clouds,-33.9258,18.4232,"Radisson Blu Hotel Waterfront, Cape Town"


In [61]:
hotel_df.isnull().sum()

City                   0
Country                0
Max Temp               0
Current Description    0
Lat                    0
Lng                    0
Hotel Name             0
dtype: int64

In [ ]:
# There is not null values in this hotel searches

In [ ]:
# 7. Drop the rows where there is no Hotel Name.
#hotel_df.dropna()

In [80]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
#city_data_df.to_csv(output_data_file, index_label= "City_ID")
hotel_df.to_csv(output_data_file, index_label= "City_ID")

In [74]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """

<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [75]:
hotel_info

['\n\n<dl>\n<dt>Hotel Name</dt><dd>Hotel Arena Blanca</dd>\n<dt>City</dt><dd>San Andres</dd>\n<dt>Country</dt><dd>CO</dd>\n<dt>Current Description</dt><dd>scattered clouds</dd>\n<dt>Max Temp</dt><dd>82.42 °F</dd>\n</dl>\n',
 '\n\n<dl>\n<dt>Hotel Name</dt><dd>Bredasdorp Country Manor</dd>\n<dt>City</dt><dd>Bredasdorp</dd>\n<dt>Country</dt><dd>ZA</dd>\n<dt>Current Description</dt><dd>overcast clouds</dd>\n<dt>Max Temp</dt><dd>64.72 °F</dd>\n</dl>\n',
 '\n\n<dl>\n<dt>Hotel Name</dt><dd>Hampton Inn & Suites Jamestown</dd>\n<dt>City</dt><dd>Jamestown</dd>\n<dt>Country</dt><dd>US</dd>\n<dt>Current Description</dt><dd>broken clouds</dd>\n<dt>Max Temp</dt><dd>72.95 °F</dd>\n</dl>\n',
 '\n\n<dl>\n<dt>Hotel Name</dt><dd>Hotel Castelo</dd>\n<dt>City</dt><dd>Cidreira</dd>\n<dt>Country</dt><dd>BR</dd>\n<dt>Current Description</dt><dd>broken clouds</dd>\n<dt>Max Temp</dt><dd>66.81 °F</dd>\n</dl>\n',
 '\n\n<dl>\n<dt>Hotel Name</dt><dd>Radisson Blu Hotel Waterfront, Cape Town</dd>\n<dt>City</dt><dd>Ca

In [76]:
locations.head()

,Lat,Lng
0,12.5847,-81.7006
1,-34.5322,20.0403
2,42.0970,-79.2353
3,-30.1811,-50.2056
4,-33.9258,18.4232


In [77]:
# 11a. Add a marker layer for each city to the map. 

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# 11b. Display the figure
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))

# End